In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.models import Sequential
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.layers import Dense, Embedding, LSTM
from keras import optimizers
from keras.models import load_model
import tensorflow_hub as hub
import bert
from bert import optimization, run_classifier, tokenization
from matplotlib import pyplot
import tf_metrics
from scipy.special import softmax

import emoji
import json, argparse, os
import re
import io
import sys

Using TensorFlow backend.
W0606 19:48:05.106102 4619879872 __init__.py:56] Some hub symbols are not available because TensorFlow version is less than 1.14


In [2]:
np.random.seed(7)

global trainDataPath, testDataPath, solutionPath, gloveDir
global NUM_FOLDS, NUM_CLASSES, MAX_NB_WORDS, MAX_SEQUENCE_LENGTH, EMBEDDING_DIM
global BATCH_SIZE, LSTM_DIM, DROPOUT, NUM_EPOCHS, LEARNING_RATE, EARLY_STOPPING

#parser = argparse.ArgumentParser(description="Baseline Script for SemEval")
#parser.add_argument('-config', help='Config to read details', required=True)
#args = parser.parse_args()

with open('testBaseline.config') as configfile:
    config = json.load(configfile)

trainDataPath = config["train_data_path"]
validationDataPath = config["validation_data_path"]
testDataPath = config["test_data_path"]
solutionPath = config["solution_path"]
gloveDir = config["glove_dir"]

NUM_FOLDS = config["num_folds"]
NUM_CLASSES = config["num_classes"]
MAX_NB_WORDS = config["max_nb_words"]
MAX_SEQUENCE_LENGTH = config["max_sequence_length"]
EMBEDDING_DIM = config["embedding_dim"]
BATCH_SIZE = config["batch_size"]
LSTM_DIM = config["lstm_dim"]
DROPOUT = config["dropout"]
LEARNING_RATE = config["learning_rate"]
NUM_EPOCHS = config["num_epochs"]
EARLY_STOPPING = config["early_stopping"]
label2emotion = {0:"others", 1:"happy", 2: "sad", 3:"angry"}
emotion2label = {"others":0, "happy":1, "sad":2, "angry":3}

In [3]:
def preprocessData(dataFilePath):
    """Load data from a file, process and return indices, conversations and labels in separate lists
    Input:
        dataFilePath : Path to train/test file to be processed
        mode : "train" mode returns labels. "test" mode doesn't return labels.
    Output:
        indices : Unique conversation ID list
        conversations : List of 3 turn conversations, processed and each turn separated by the <eos> tag
        labels : [Only available in "train" mode] List of labels
    """
    data = {}
    data['conversations1'] = []
    data['conversations2'] = []
    data['labels'] = []
    with io.open(dataFilePath, encoding="utf8") as finput:
        finput.readline()
        for line in finput:
            line = line.replace('\n','').split('\t')
            label = emotion2label[line[4]]
            conv1 = ' <eos> '.join(line[1:3])
            conv2 = line[3]
            data['conversations1'].append(conv1)
            data['conversations2'].append(conv2)
            data['labels'].append(label)
    return pd.DataFrame.from_dict(data)
trainData = preprocessData(trainDataPath)
validationData = preprocessData(validationDataPath)
#trainData = pd.concat([trainData,validationData])
testData = preprocessData(testDataPath)

In [4]:
testingData = preprocessData('testing.txt')

In [5]:
trainData.head()

,conversations1,conversations2,labels
0,Don't worry I'm girl <eos> hmm how do I know ...,What's ur name?,0
1,When did I? <eos> saw many times i think -_-,No. I never saw you,3
2,By <eos> by Google Chrome,Where you live,0
3,U r ridiculous <eos> I might be ridiculous but...,U little disgusting whore,3
4,Just for time pass <eos> wt do u do 4 a living...,Maybe,0


In [6]:
os.getcwd()

'/Users/nathanieljoselson/Documents/Projects/masters/natemusMasters/bert_F1_69'

In [7]:
DATA_COLUMN1 = 'conversations1'
DATA_COLUMN2 = 'conversations2'
LABEL_COLUMN = 'labels'
OUTPUT_DIR = os.getcwd()
# label_list is the list of labels, i.e. True, False or 0, 1 or 'dog', 'cat'
label_list = [0, 1, 2, 3]

In [8]:
# Use the InputExample class from BERT's run_classifier code to create examples from the data
train_InputExamples = trainData.apply(lambda x: bert.run_classifier.InputExample(guid=None, # Globally unique ID for bookkeeping, unused in this example
                                                                   text_a = x[DATA_COLUMN1], 
                                                                   text_b = x[DATA_COLUMN2], 
                                                                   label = x[LABEL_COLUMN]), axis = 1)

test_InputExamples = testData.apply(lambda x: bert.run_classifier.InputExample(guid=None, 
                                                                   text_a = x[DATA_COLUMN1], 
                                                                   text_b = x[DATA_COLUMN2], 
                                                                   label = x[LABEL_COLUMN]), axis = 1)

In [9]:
# This is a path to an uncased (all lowercase) version of BERT
BERT_MODEL_HUB = "https://tfhub.dev/google/bert_uncased_L-12_H-768_A-12/1"

def create_tokenizer_from_hub_module():
    """Get the vocab file and casing info from the Hub module."""
    with tf.Graph().as_default():
        bert_module = hub.Module(BERT_MODEL_HUB)
        tokenization_info = bert_module(signature="tokenization_info", as_dict=True)
        with tf.Session() as sess:
            vocab_file, do_lower_case = sess.run([tokenization_info["vocab_file"],tokenization_info["do_lower_case"]])
      
    return bert.tokenization.FullTokenizer(
        vocab_file=vocab_file, do_lower_case=do_lower_case)

tokenizer = create_tokenizer_from_hub_module()

Instructions for updating:
Colocations handled automatically by placer.


W0606 19:49:13.841689 4619879872 deprecation.py:323] From /Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/tensorflow/python/ops/control_flow_ops.py:3632: colocate_with (from tensorflow.python.framework.ops) is deprecated and will be removed in a future version.
Instructions for updating:
Colocations handled automatically by placer.


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


I0606 19:49:15.693947 4619879872 saver.py:1483] Saver not created because there are no variables in the graph to restore


In [10]:
# We'll set sequences to be at most 128 tokens long.
MAX_SEQ_LENGTH = 128
# Convert our train and test features to InputFeatures that BERT understands.
train_features = bert.run_classifier.convert_examples_to_features(train_InputExamples, label_list, MAX_SEQ_LENGTH, tokenizer)
test_features = bert.run_classifier.convert_examples_to_features(test_InputExamples, label_list, MAX_SEQ_LENGTH, tokenizer)

INFO:tensorflow:Writing example 0 of 30160


I0606 19:49:16.200010 4619879872 run_classifier.py:774] Writing example 0 of 30160


INFO:tensorflow:*** Example ***


I0606 19:49:16.209877 4619879872 run_classifier.py:461] *** Example ***


INFO:tensorflow:guid: None


...

I0606 19:49:25.823215 4619879872 run_classifier.py:466] input_mask: 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0


INFO:tensorflow:segment_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0


I0606 19:49:25.825095 4619879872 run_classifier.py:467] segment_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0


INFO:tensorflow:label: 0 (id = 0)


I0606 19:49:25.826480 4619879872 run_classifier.py:468] label: 0 (id = 0)


In [11]:
def create_model(is_predicting, input_ids, input_mask, segment_ids, labels,
                 num_labels):
    """Creates a classification model."""

    bert_module = hub.Module(
        BERT_MODEL_HUB,
        trainable=True)
    bert_inputs = dict(
        input_ids=input_ids,
        input_mask=input_mask,
        segment_ids=segment_ids)
    bert_outputs = bert_module(
        inputs=bert_inputs,
        signature="tokens",
        as_dict=True)

    # Use "pooled_output" for classification tasks on an entire sentence.
    # Use "sequence_outputs" for token-level output.
    output_layer = bert_outputs["pooled_output"]

    hidden_size = output_layer.shape[-1].value

    # Create our own layer to tune for politeness data.
    output_weights = tf.get_variable(
        "output_weights", [num_labels, hidden_size],
        initializer=tf.truncated_normal_initializer(stddev=0.02))

    output_bias = tf.get_variable(
        "output_bias", [num_labels], initializer=tf.zeros_initializer())

    with tf.variable_scope("loss"):

        # Dropout helps prevent overfitting
        output_layer = tf.nn.dropout(output_layer, keep_prob=0.9)
  
        logits = tf.matmul(output_layer, output_weights, transpose_b=True)
        logits = tf.nn.bias_add(logits, output_bias)
        log_probs = tf.nn.log_softmax(logits, axis=-1)

        # Convert labels into one-hot encoding
        one_hot_labels = tf.one_hot(labels, depth=num_labels, dtype=tf.float32)

        predicted_labels = tf.squeeze(tf.argmax(log_probs, axis=-1, output_type=tf.int32))
        # If we're predicting, we want predicted labels and the probabiltiies.
        if is_predicting:
            return (predicted_labels, log_probs)

        # If we're train/eval, compute loss between predicted and actual label
        per_example_loss = -tf.reduce_sum(one_hot_labels * log_probs, axis=-1)
        loss = tf.reduce_mean(per_example_loss)
        return (loss, predicted_labels, log_probs)


In [12]:
# model_fn_builder actually creates our model function
# using the passed parameters for num_labels, learning_rate, etc.
def model_fn_builder(num_labels, learning_rate, num_train_steps,
                     num_warmup_steps):
    """Returns `model_fn` closure for TPUEstimator."""
    def model_fn(features, labels, mode, params):  # pylint: disable=unused-argument
        """The `model_fn` for TPUEstimator."""

        input_ids = features["input_ids"]
        input_mask = features["input_mask"]
        segment_ids = features["segment_ids"]
        label_ids = features["label_ids"]

        is_predicting = (mode == tf.estimator.ModeKeys.PREDICT)

        # TRAIN and EVAL
        if not is_predicting:

            (loss, predicted_labels, log_probs) = create_model(is_predicting, input_ids, input_mask, segment_ids, label_ids, num_labels)

            train_op = bert.optimization.create_optimizer(loss, learning_rate, num_train_steps, num_warmup_steps, use_tpu=False)

            # Calculate evaluation metrics. 
            def metric_fn(label_ids, predicted_labels):
                accuracy = tf.metrics.accuracy(label_ids, predicted_labels)
                recall = tf.metrics.recall(
                    label_ids,
                    predicted_labels)
                precision = tf.metrics.precision(
                    label_ids,
                    predicted_labels)
                return {
                    "eval_accuracy": accuracy,
                    "precision": precision,
                    "recall": recall,
                }

            eval_metrics = metric_fn(label_ids, predicted_labels)

            if mode == tf.estimator.ModeKeys.TRAIN:
                return tf.estimator.EstimatorSpec(mode=mode,loss=loss,train_op=train_op)
            else:
                return tf.estimator.EstimatorSpec(mode=mode,
                    loss=loss,
                    eval_metric_ops=eval_metrics)
        else:
            (predicted_labels, log_probs) = create_model(is_predicting, input_ids, input_mask, segment_ids, label_ids, num_labels)

            predictions = {'probabilities': log_probs,'labels': predicted_labels
              }
            return tf.estimator.EstimatorSpec(mode, predictions=predictions)

    # Return the actual model function in the closure
    return model_fn


In [13]:
# Compute train and warmup steps from batch size
# These hyperparameters are copied from this colab notebook (https://colab.sandbox.google.com/github/tensorflow/tpu/blob/master/tools/colab/bert_finetuning_with_cloud_tpus.ipynb)
BATCH_SIZE = 32
LEARNING_RATE = 2e-5
NUM_TRAIN_EPOCHS = 3.0
# Warmup is a period of time where hte learning rate 
# is small and gradually increases--usually helps training.
WARMUP_PROPORTION = 0.1
# Model configs
SAVE_CHECKPOINTS_STEPS = 100
SAVE_SUMMARY_STEPS = 100

In [14]:
# Compute # train and warmup steps from batch size
num_train_steps = int(len(train_features) / BATCH_SIZE * NUM_TRAIN_EPOCHS)
num_warmup_steps = int(num_train_steps * WARMUP_PROPORTION)

In [15]:
# Specify outpit directory and number of checkpoint steps to save
run_config = tf.estimator.RunConfig(
    model_dir=OUTPUT_DIR,
    save_summary_steps=SAVE_SUMMARY_STEPS,
    save_checkpoints_steps=SAVE_CHECKPOINTS_STEPS)

In [16]:
model_fn = model_fn_builder(
    num_labels=len(label_list),
    learning_rate=LEARNING_RATE,
    num_train_steps=num_train_steps,
    num_warmup_steps=num_warmup_steps)

estimator = tf.estimator.Estimator(
    model_fn=model_fn,
    config=run_config,
    params={"batch_size": BATCH_SIZE})


INFO:tensorflow:Using config: {'_model_dir': '/Users/nathanieljoselson/Documents/Projects/masters/natemusMasters/bert_F1_69', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 100, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x137c42b00>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


I0606 19:49:27.767205 4619879872 estimator.py:201] Using config: {'_model_dir': '/Users/nathanieljoselson/Documents/Projects/masters/natemusMasters/bert_F1_69', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 100, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x137c42b00>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [17]:
# Create an input function for training. drop_remainder = True for using TPUs.
train_input_fn = bert.run_classifier.input_fn_builder(
    features=train_features,
    seq_length=MAX_SEQ_LENGTH,
    is_training=True,
    drop_remainder=False)

In [18]:
estimator.train(input_fn=train_input_fn, max_steps=num_train_steps)

INFO:tensorflow:Skipping training since max_steps has already saved.


I0606 19:49:27.834203 4619879872 estimator.py:351] Skipping training since max_steps has already saved.


In [19]:
test_input_fn = run_classifier.input_fn_builder(
    features=test_features,
    seq_length=MAX_SEQ_LENGTH,
    is_training=False,
    drop_remainder=False)

In [20]:
estimator.evaluate(input_fn=test_input_fn, steps=None)

INFO:tensorflow:Calling model_fn.


I0606 19:49:30.472592 4619879872 estimator.py:1111] Calling model_fn.


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


I0606 19:49:33.469495 4619879872 saver.py:1483] Saver not created because there are no variables in the graph to restore


Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


...

INFO:tensorflow:Saving dict for global step 3085: eval_accuracy = 0.90506446, global_step = 3085, loss = 0.35250324, precision = 0.6766699, recall = 0.8401442


I0606 20:24:01.151319 4619879872 estimator.py:1979] Saving dict for global step 3085: eval_accuracy = 0.90506446, global_step = 3085, loss = 0.35250324, precision = 0.6766699, recall = 0.8401442


INFO:tensorflow:Saving 'checkpoint_path' summary for global step 3085: /Users/nathanieljoselson/Documents/Projects/masters/natemusMasters/bert_F1_69/model.ckpt-3085


I0606 20:24:04.742187 4619879872 estimator.py:2039] Saving 'checkpoint_path' summary for global step 3085: /Users/nathanieljoselson/Documents/Projects/masters/natemusMasters/bert_F1_69/model.ckpt-3085


{'eval_accuracy': 0.90506446,
 'loss': 0.35250324,
 'precision': 0.6766699,
 'recall': 0.8401442,
 'global_step': 3085}

In [21]:
def getPrediction(in_sentences,verbose):
    labels = ["others", "happy", "sad", "angry"]
    input_examples = [run_classifier.InputExample(guid="", text_a = x[0], text_b = x[1], label = 0) for x in in_sentences] # here, "" is just a dummy label
    input_features = run_classifier.convert_examples_to_features(input_examples, label_list, MAX_SEQ_LENGTH, tokenizer)
    predict_input_fn = run_classifier.input_fn_builder(features=input_features, seq_length=MAX_SEQ_LENGTH, is_training=False, drop_remainder=False)
    predictions = estimator.predict(predict_input_fn)
    if verbose:
        return [(sentence, prediction['probabilities'], labels[prediction['labels']]) for sentence, prediction in zip(in_sentences, predictions)]
    else:
        return [prediction['probabilities'] for sentence, prediction in zip(in_sentences, predictions)]

In [50]:
pred_sentences = [
  ["That movie was absolutely awful ",
  "The acting was a bit lacking"],
  ["The film was creative and surprising",
  "Absolutely fantastic!"]
]

In [51]:
preds = getPrediction(pred_sentences,True)

INFO:tensorflow:Writing example 0 of 2


I0612 07:04:44.630981 4619879872 run_classifier.py:774] Writing example 0 of 2


INFO:tensorflow:*** Example ***


I0612 07:04:44.661104 4619879872 run_classifier.py:461] *** Example ***


INFO:tensorflow:guid: 


...

INFO:tensorflow:label: 0 (id = 0)


I0612 07:04:44.726172 4619879872 run_classifier.py:468] label: 0 (id = 0)


INFO:tensorflow:Calling model_fn.


I0612 07:04:46.149296 4619879872 estimator.py:1111] Calling model_fn.


RuntimeError: Missing implementation that supports: loader(*('/var/folders/fn/6m7f8mm57zv69xbwd8751w_r0000gn/T/tfhub_modules/5a395eafef2a37bd9fc55d7f6ae676d2a134a838',), **{})

In [49]:
preds

[(['That movie was absolutely awful', 'The acting was a bit lacking'],
  array([-1.3564274 , -2.333234  , -0.45688406, -4.4075212 ], dtype=float32),
  'sad'),
 (['The film was creative and surprising', 'Absolutely fantastic!'],
  array([-4.5314517 , -0.01164143, -8.093053  , -7.594468  ], dtype=float32),
  'happy')]

In [25]:
#pred_train = trainData[['conversations1','conversations2']].values.tolist()
pred_val = validationData[['conversations1','conversations2']].values.tolist()
pred_test = testData[['conversations1','conversations2']].values.tolist()
#predictions_train = getPrediction(pred_train,False)
predictions_val = getPrediction(pred_val,False)
predictions_test = getPrediction(pred_test,False)

INFO:tensorflow:Writing example 0 of 2755


I0606 20:24:12.500034 4619879872 run_classifier.py:774] Writing example 0 of 2755


INFO:tensorflow:*** Example ***


I0606 20:24:12.502298 4619879872 run_classifier.py:461] *** Example ***


INFO:tensorflow:guid: 


...

I0606 20:41:05.236994 4619879872 saver.py:1270] Restoring parameters from /Users/nathanieljoselson/Documents/Projects/masters/natemusMasters/bert_F1_69/model.ckpt-3085


INFO:tensorflow:Running local_init_op.


I0606 20:41:07.675966 4619879872 session_manager.py:491] Running local_init_op.


INFO:tensorflow:Done running local_init_op.


I0606 20:41:07.793032 4619879872 session_manager.py:493] Done running local_init_op.


In [26]:
pred_testing = testingData[['conversations1','conversations2']].values.tolist()
predictions_testing = getPrediction(pred_testing,False)

INFO:tensorflow:Writing example 0 of 8


I0606 21:24:29.329618 4619879872 run_classifier.py:774] Writing example 0 of 8


INFO:tensorflow:*** Example ***


I0606 21:24:29.333384 4619879872 run_classifier.py:461] *** Example ***


INFO:tensorflow:guid: 


...

I0606 21:24:33.880729 4619879872 saver.py:1270] Restoring parameters from /Users/nathanieljoselson/Documents/Projects/masters/natemusMasters/bert_F1_69/model.ckpt-3085


INFO:tensorflow:Running local_init_op.


I0606 21:24:36.172999 4619879872 session_manager.py:491] Running local_init_op.


INFO:tensorflow:Done running local_init_op.


I0606 21:24:36.273712 4619879872 session_manager.py:493] Done running local_init_op.


In [27]:
pd.DataFrame(pred_test,columns = ['turn1 + turn2' , 'turn3'])

,turn1 + turn2,turn3
0,Hmm <eos> What does your bio mean?,I don’t have any bio
1,What you like <eos> very little things,Ok
2,Yes <eos> How so?,I want to fuck babu
3,what did you guess <eos> what what,fuck
4,We ? <eos> of course we will!,What gender movies you like??
5,Where are you now? <eos> At home just about to...,what are you eating?
6,That was a joke btw... <eos> it was,Yes 😁
7,Who d hell s he <eos> johnny depp...duh?,Who she
8,"yes, good advice <eos> best advice ...",i great thx
9,"Nice to meet u <eos> Hi, nice to meet you too! 😸😂",😁


In [28]:
#bertpreds_train = np.asarray(predictions_train)
bertpreds_val = np.asarray(predictions_val)
bertpreds_test = np.asarray(predictions_test)
#np.savetxt("bertpreds_train.csv", bertpreds_train, delimiter=",")
np.savetxt("bertpreds_val.csv", bertpreds_val, delimiter=",")
np.savetxt("bertpreds_test.csv", bertpreds_test, delimiter=",")

In [29]:
bertpreds_testing = np.asarray(predictions_testing)

In [30]:
def getMetrics(predictions, ground):
    """Given predicted labels and the respective ground truth labels, display some metrics
    Input: shape [# of samples, NUM_CLASSES]
        predictions : Model output. Every row has 4 decimal values, with the highest belonging to the predicted class
        ground : Ground truth labels, converted to one-hot encodings. A sample belonging to Happy class will be [0, 1, 0, 0]
    Output:
        accuracy : Average accuracy
        microPrecision : Precision calculated on a micro level. Ref - https://datascience.stackexchange.com/questions/15989/micro-average-vs-macro-average-performance-in-a-multiclass-classification-settin/16001
        microRecall : Recall calculated on a micro level
        microF1 : Harmonic mean of microPrecision and microRecall. Higher value implies better classification
    """
    # [0.1, 0.3 , 0.2, 0.1] -> [0, 1, 0, 0]
    discretePredictions = to_categorical(predictions.argmax(axis=1))

    truePositives = np.sum(discretePredictions*ground, axis=0)
    falsePositives = np.sum(np.clip(discretePredictions - ground, 0, 1), axis=0)
    falseNegatives = np.sum(np.clip(ground-discretePredictions, 0, 1), axis=0)

    print("True Positives per class : ", truePositives)
    print("False Positives per class : ", falsePositives)
    print("False Negatives per class : ", falseNegatives)

    # ------------- Macro level calculation ---------------
    macroPrecision = 0
    macroRecall = 0
    # We ignore the "Others" class during the calculation of Precision, Recall and F1
    for c in range(1, NUM_CLASSES):
        precision = truePositives[c] / (truePositives[c] + falsePositives[c])
        macroPrecision += precision
        recall = truePositives[c] / (truePositives[c] + falseNegatives[c])
        macroRecall += recall
        f1 = ( 2 * recall * precision ) / (precision + recall) if (precision+recall) > 0 else 0
        print("Class %s : Precision : %.3f, Recall : %.3f, F1 : %.3f" % (label2emotion[c], precision, recall, f1))

    macroPrecision /= 3
    macroRecall /= 3
    macroF1 = (2 * macroRecall * macroPrecision ) / (macroPrecision + macroRecall) if (macroPrecision+macroRecall) > 0 else 0
    print("Ignoring the Others class, Macro Precision : %.4f, Macro Recall : %.4f, Macro F1 : %.4f" % (macroPrecision, macroRecall, macroF1))

    # ------------- Micro level calculation ---------------
    truePositives = truePositives[1:].sum()
    falsePositives = falsePositives[1:].sum()
    falseNegatives = falseNegatives[1:].sum()

    print("Ignoring the Others class, Micro TP : %d, FP : %d, FN : %d" % (truePositives, falsePositives, falseNegatives))

    microPrecision = truePositives / (truePositives + falsePositives)
    microRecall = truePositives / (truePositives + falseNegatives)

    microF1 = ( 2 * microRecall * microPrecision ) / (microPrecision + microRecall) if (microPrecision+microRecall) > 0 else 0
    # -----------------------------------------------------

    predictions = predictions.argmax(axis=1)
    ground = ground.argmax(axis=1)
    accuracy = np.mean(predictions==ground)

    print("Accuracy : %.4f, Micro Precision : %.4f, Micro Recall : %.4f, Micro F1 : %.4f" % (accuracy, microPrecision, microRecall, microF1))
    return accuracy, microPrecision, microRecall, microF1

In [31]:
testLabels = to_categorical(np.asarray(testData.labels.to_list()))

In [32]:
accuracy, microPrecision, microRecall, microF1 = getMetrics(np.asarray(predictions_test), testLabels)

True Positives per class :  [4343.  193.  199.  253.]
False Positives per class :  [133. 140. 105. 143.]
False Negatives per class :  [334.  91.  51.  45.]
Class happy : Precision : 0.580, Recall : 0.680, F1 : 0.626
Class sad : Precision : 0.655, Recall : 0.796, F1 : 0.718
Class angry : Precision : 0.639, Recall : 0.849, F1 : 0.729
Ignoring the Others class, Macro Precision : 0.6244, Macro Recall : 0.7749, Macro F1 : 0.6915
Ignoring the Others class, Micro TP : 645, FP : 388, FN : 187
Accuracy : 0.9054, Micro Precision : 0.6244, Micro Recall : 0.7752, Micro F1 : 0.6917


In [33]:
valLabels = to_categorical(np.asarray(validationData.labels.to_list()))

In [34]:
accuracy, microPrecision, microRecall, microF1 = getMetrics(np.asarray(predictions_val), valLabels)

True Positives per class :  [2261.  124.  114.  145.]
False Positives per class :  [21. 43. 25. 22.]
False Negatives per class :  [77. 18. 11.  5.]
Class happy : Precision : 0.743, Recall : 0.873, F1 : 0.803
Class sad : Precision : 0.820, Recall : 0.912, F1 : 0.864
Class angry : Precision : 0.868, Recall : 0.967, F1 : 0.915
Ignoring the Others class, Macro Precision : 0.8103, Macro Recall : 0.9173, Macro F1 : 0.8605
Ignoring the Others class, Micro TP : 383, FP : 90, FN : 34
Accuracy : 0.9597, Micro Precision : 0.8097, Micro Recall : 0.9185, Micro F1 : 0.8607


In [35]:
dat = list(zip(testData.conversations.to_list(),np.asarray(predictions).argmax(axis=1),testData.labels.to_list()))
df = pd.DataFrame(dat, columns=['Conversation','Predicted','True'])

AttributeError: 'DataFrame' object has no attribute 'conversations'

In [ ]:
df.to_csv('predictions.csv')

In [38]:
testData = pd.read_csv(testDataPath, sep='\t')
discretePredictions = to_categorical(bertpreds_test.argmax(axis=1))
pred_emotion = []
probabilities = []
for pred in bertpreds_test:
    emotion = label2emotion[np.argmax(pred)]
    prob = [ '%.2f' % elem for elem in pred ]
    probabilities.append(prob)
    pred_emotion.append(emotion)    
testData['predictions'] = pred_emotion
testData['probabilities'] = probabilities
pd.set_option('display.max_rows', 6000)

def softmax_(col):
    return softmax(list(map(np.float16,col)))
    
testData['Probs'] = testData['probabilities'].apply(softmax_)
testData[['others','happy','sad','angry']] = pd.DataFrame(testData.Probs.values.tolist(), index= testData.index)

In [43]:
testData[(testData.label==testData.predictions)&(testData.label=='angry')][['turn1','turn2','turn3','label','others','happy','sad','angry']]

,turn1,turn2,turn3,label,others,happy,sad,angry
13,First you hurt me,okay,So I talked rude,angry,0.002449,0.000777,0.178955,0.817871
29,Uuu,no i'n not !! what's wrong with you ?!?!,I hate uuu,angry,0.001527,0.000388,0.002329,0.996094
61,I dont know,-cries- 😦😧,Fuck off,angry,0.039093,0.000500,0.000860,0.959473
64,I get success in everything i cnat get success...,My anger controls me.,What i csnt so please get away and don't mean ...,angry,0.003706,0.000455,0.001033,0.995117
84,Where,very much! ;) :p,Ok bye dont text me again i am angry with u,angry,0.000756,0.000699,0.005478,0.993164
109,I won't,haha I know you to well!!,Go to hell,angry,0.001945,0.000856,0.001412,0.996094
130,Okkkkkkkk please block me,don't tell me what to do,I want fuck you,angry,0.005585,0.000278,0.001062,0.993164
176,So I'm deleting you right now,I think so.,Buy stupid,angry,0.001690,0.000437,0.001108,0.997070
197,Yup,Thanks.,You are so annoying,angry,0.003510,0.000289,0.001014,0.995117
232,Name me the feelings that is in your program,Me : What's your name? Autowala : NOshad Me : ...,What the hell. Lame stuff,angry,0.004349,0.000492,0.000827,0.994141


In [44]:
testingData = pd.read_csv('testing.txt', sep='\t')
discretePredictions = to_categorical(bertpreds_testing.argmax(axis=1))
pred_emotion = []
probabilities = []
for pred in bertpreds_testing:
    emotion = label2emotion[np.argmax(pred)]
    prob = [ '%.2f' % elem for elem in pred ]
    probabilities.append(prob)
    pred_emotion.append(emotion)    
testingData['predictions'] = pred_emotion
testingData['probabilities'] = probabilities
pd.set_option('display.max_rows', 6000)

def softmax_(col):
    return softmax(list(map(np.float16,col)))
    
testingData['Probs'] = testingData['probabilities'].apply(softmax_)
testingData[['others','happy','sad','angry']] = pd.DataFrame(testingData.Probs.values.tolist(), index= testingData.index)

In [46]:
testingData[['turn1','turn2','turn3','label','others','happy','sad','angry']]

,turn1,turn2,turn3,label,others,happy,sad,angry
0,You are such a good friend,Thanks!,😁,happy,0.216064,0.769531,0.013245,0.001557
1,Why don’t you ever write me back?,Cause you’re a creep.,I know... I am... 😔,sad,0.021545,0.043335,0.916016,0.019089
2,I don’t get it… why can’t I ever get a date?,🙃 too ugly?,Screw you!,angry,0.003195,0.000492,0.001842,0.994141
3,Too much pollution in the world,Or too many people…,Or both…,others,0.997070,0.000347,0.001166,0.000926
4,You are such a good friend,Thanks!,I really fucking mean it! 😁,happy,0.011192,0.029785,0.021210,0.937988
5,Why don’t you ever write me back?,Cause you’re a creep.,I know... I am...,sad,0.450195,0.000536,0.010353,0.539062
6,I don’t get it… why can’t I ever get a date?,🙃 too ugly?,Screw you! 😭,angry,0.004318,0.006111,0.017136,0.972656
7,Too much pollution in the world 😭,Or too many people…,Or both…,others,0.833008,0.015854,0.146362,0.004929
